In [ ]:
import tensorflow as tf

tf.config.get_visible_devices()

In [ ]:
from core.trainer import ModelParams
single_model = ModelParams(
        sf_model = ["single_vertex_model"], # Input layer: 5 * (2 * spin  + 1), Output layer: forward = 5 + 1, backward = 5
        spin_j = [3.5, 6.5],
        main_layer_hidden_nodes = [(64, 32, 16, 8), (64, 32, 32), (64, 16), (64, 64, 16, 16)],
        branch1_hidden_nodes = [()],
        branch2_hidden_nodes = [()],
        activation = ["swish", "tanh", "relu"],
        exploration_rate = [0.5],
        training_fraction_from_back_traj = [0.0],
        learning_rate = [0.0005],
        batch_size = [int(1e3)],
        n_iterations = [int(1e4)],
        evaluation_batch_size = [int(1e6)],
        generate_samples_every_m_training_samples = [int(1e6)],
)

star_model = ModelParams(
        sf_model = ["star_model"], # Input layer: 20 * (2 * spin  + 1), Output layer: forward = 20 + 1, backward = 20
        spin_j = [3.5, 6.5],
        main_layer_hidden_nodes = [(256, 128, 64, 32), (256, 64, 64, 32), (256, 32), (256, 192, 64, 32)],
        branch1_hidden_nodes = [()],
        branch2_hidden_nodes = [()],
        activation = ["swish", "tanh", "relu"],
        exploration_rate = [0.5],
        training_fraction_from_back_traj = [1.0],
        learning_rate = [0.0005],
        batch_size = [int(1e3)],
        n_iterations = [int(1e4)],
        evaluation_batch_size = [int(1e6)],
        generate_samples_every_m_training_samples = [int(1e6)],
)

In [ ]:
from core.trainer import test_models

test_models(single_model)

models = [single_model, star_model]

total_number_of_models = sum(map(len, models))
print(f"Total number of models: {total_number_of_models} to run.")
print("Estimated time to run (5 mins per model):", total_number_of_models * 5 / 60, "hours")

fields = single_model.fields()

for i, params in enumerate(single_model):
    print(i, params)
    if i > 4:
        break

from core.trainer import train_gfn
from dataclasses import dataclass
import itertools

import os
import tensorflow as tf

import datetime
import pickle

single_model = ModelParams(
        sf_model = ["single_vertex_model"], # Input layer: 5 * (2 * spin  + 1), Output layer: forward = 5 + 1, backward = 5
        spin_j = [3.5, 6.5],
        main_layer_hidden_nodes = [(64, 32, 16, 8), (64, 32, 32), (64, 16), (64, 64, 16, 16)],
        branch1_hidden_nodes = [()],
        branch2_hidden_nodes = [()],
        activation = ["swish", "tanh", "relu"],
        exploration_rate = [0.5],
        training_fraction_from_back_traj = [0.0],
        learning_rate = [0.0005],
        batch_size = [int(1e1)],
        n_iterations = [int(1e1)],
        evaluation_batch_size = [int(1e1)],
        generate_samples_every_m_training_samples = [int(1e3)],
)

star_model = ModelParams(
        sf_model = ["star_model"], # Input layer: 20 * (2 * spin  + 1), Output layer: forward = 20 + 1, backward = 20
        spin_j = [3.5, 6.5],
        main_layer_hidden_nodes = [(256, 128, 64, 32), (256, 64, 64, 32), (256, 32), (256, 192, 64, 32)],
        branch1_hidden_nodes = [()],
        branch2_hidden_nodes = [()],
        activation = ["swish", "tanh", "relu"],
        exploration_rate = [0.5],
        training_fraction_from_back_traj = [1.0],
        learning_rate = [0.0005],
        batch_size = [int(1e1)],
        n_iterations = [int(1e1)],
        evaluation_batch_size = [int(1e1)],
        generate_samples_every_m_training_samples = [int(1e3)],
)

models = [single_model, star_model]

total_number_of_models = sum(map(len, models))
total_number_of_models = sum(map(len, models))
tf.print(f"Total number of models:", total_number_of_models)
tf.print(f"Expected time to complete (5 mins per model):", total_number_of_models * 5 / 60, "hours")

models_avg_losses = {
    model.sf_model[0]: []
    for model in models
}

start = datetime.datetime.now()
tf.print("Starting testing:", datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "\n")

os.makedirs("results", exist_ok=True)
# import numpy as np
for model in models:
    num_models = len(model)
    tf.print("Testing model:", model.sf_model[0], "\n")
    with open(f"results/{model.sf_model[0]}-params.pkl", "wb") as f:
        f.write(b"training_time, params, avg_losses\n")
    for i, params in enumerate(model):
        tf.print(f"Starting training for parameter set {i} of {num_models}")
        training_start = datetime.datetime.now()
        avg_losses = train_gfn(**params)
        # avg_losses = np.random.rand(10)
        training_time = (datetime.datetime.now() - training_start).total_seconds()
        tf.print("Finished training, elapsed time:", training_time / 60, "minutes\n")
        models_avg_losses[model.sf_model[0]].append((training_time, avg_losses, params))

        with open(f"results/{model.sf_model[0]}-params.pkl", "ab") as f:
            pickle.dump((training_time, params, avg_losses), f)


from pathlib import Path
import os
import pickle

with open("results/single_vertex_model-params.pkl", "rb") as f:
    header = f.readline()
    while True:
        try:
            data = pickle.load(f)
            print(data)
        except EOFError:
            break